In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mne
import glob

%matplotlib notebook

In [2]:
subject_id = 'S01'
alpha_dir = '/Users/nicolaspiron/Documents/PULSATION/Data/EEG/Alpheye/'
files = glob.glob(alpha_dir + subject_id +'*')
#sorting files because they are sometime in the wrong order
files.sort()

raw = mne.io.read_raw_bdf(files[1])
raw.load_data()

raw.plot(scalings='auto');
raw.info

Extracting EDF parameters from /Users/nicolaspiron/Documents/PULSATION/Data/EEG/Alpheye/S01_Alpheye_r2.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 357375  =      0.000 ...   348.999 secs...
Using qt as 2D backend.
Using pyopengl with version 3.1.6


<Info | 7 non-empty values
 bads: []
 ch_names: Fp1, AF7, AF3, F1, F3, F5, F7, FT7, FC5, FC3, FC1, C1, C3, C5, ...
 chs: 72 EEG, 1 Stimulus
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 208.0 Hz
 meas_date: 2022-11-14 15:45:30 UTC
 nchan: 73
 projs: []
 sfreq: 1024.0 Hz
>

Channels marked as bad:
none


In [3]:
#Resample, filter, interpolate

raw.drop_channels(['EXG7','EXG8'])

raw.set_channel_types({'EXG1': 'eog', 'EXG2': 'eog','EXG3': 'eog','EXG4': 'eog', 'EXG5': 'eog','EXG6': 'eog'})

raw.set_montage('biosemi64') 
#raw.plot_sensors(show_names=True);
raw.resample(500)
raw.filter(1.,30., phase='zero-double')

#interpolate
bad_channels = ['F3', 'FT7', 'F4', 'FC6']
raw.info['bads'] = bad_channels
mne.io.Raw.interpolate_bads(raw, reset_bads=True)
raw.plot()

Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
90 events found
Event IDs: [  2   4  10 128 129]
Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
90 events found
Event IDs: [  2   4  10 128 129]
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-12 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-12 dB cutoff frequency: 33.75 Hz)
- Filter length: 1651 sample

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Interpolating bad channels
    Automatic origin fit: head of radius 95.0 mm
Computing interpolation matrix from 60 sensor positions
Interpolating 4 sensors


[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    0.4s finished


Using pyopengl with version 3.1.6


Channels marked as bad:
none


In [5]:
#Compute ICA

from mne.preprocessing import ICA

ica = ICA(n_components=24, method='infomax', random_state=42)
ica.fit(raw, decim=10)

Fitting ICA to data using 64 channels (please be patient, this may take a while)
Selecting by number: 24 components
 
Fitting ICA took 2.9s.


Method,infomax
Fit,500 iterations on raw data (17450 samples)
ICA components,24
Available PCA components,64
Channel types,eeg
ICA components marked for exclusion,—


In [ ]:
#Plot components

ica.plot_components()

In [6]:
#Components choice based on visualization

ica.exclude = [0]
raw_ica= ica.apply(raw)

Applying ICA to Raw instance
    Transforming to ICA space (24 components)
    Zeroing out 1 ICA component
    Projecting back using 64 PCA components


In [7]:
#Create epochs

e_list = mne.find_events(raw_ica, stim_channel='Status')
df = pd.DataFrame(e_list, columns=['timepoint', 'duration', 'stim'])
df_stim = df[(df['stim'] == 2) | (df['stim'] == 4)].reset_index()
df_stim = df_stim.add_prefix('img_')

mne_events = df_stim[['img_timepoint', 'img_duration', 'img_stim']].values
event_dict = {'Landscape':2,
             'Human':4}

epochs = mne.Epochs(raw_ica, mne_events, event_id=event_dict, tmin=-0.2, tmax=6, baseline=(-0.2, 0), event_repeated='drop', preload=True)
epochs.set_eeg_reference(ref_channels='average');

Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
90 events found
Event IDs: [  2   4  10 128 129]
Not setting metadata
30 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 30 events and 3101 original time points ...
0 bad epochs dropped
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


In [8]:
e_list

array([[ 22565,      0,      4],
       [ 25568,      0,     10],
       [ 26637,      0,    128],
       [ 28021,      0,      4],
       [ 31024,      0,     10],
       [ 31266,      0,    128],
       [ 32692,      0,      4],
       [ 35695,      0,     10],
       [ 36004,      0,    128],
       [ 37639,      0,      4],
       [ 40642,      0,     10],
       [ 40850,      0,    128],
       [ 42360,      0,      2],
       [ 45363,      0,     10],
       [ 45747,      0,    128],
       [ 47515,      0,      4],
       [ 50518,      0,     10],
       [ 50751,      0,    128],
       [ 52236,      0,      4],
       [ 55239,      0,     10],
       [ 55582,      0,    128],
       [ 57266,      0,      4],
       [ 60269,      0,     10],
       [ 60478,      0,    128],
       [ 62221,      0,      4],
       [ 65224,      0,     10],
       [ 65533,      0,    128],
       [ 67109,      0,      4],
       [ 70112,      0,     10],
       [ 70312,      0,    128],
       [ 7

In [ ]:
#Save files (don't forget to change the 'run' variable!!)

run = '_r2'

out_dir = '/Users/nicolaspiron/Documents/PULSATION/Python_MNE/preproc/Alpheye_out/data'
ICA_filename = subject_id+run+'_ICA.fif'
raw_filename = subject_id+run+'_raw.fif'
raw_ICA_filename = subject_id+run+'_rawICA.fif'

ica.save(os.path.join(out_dir, 'ica', ICA_filename), overwrite=True)
raw.save(os.path.join(out_dir, raw_filename), overwrite=True)
raw_ica.save(os.path.join(out_dir, 'raw_ica', raw_ICA_filename), overwrite=True)